# Задание 1.1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [109]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [110]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [111]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [112]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

In [113]:
# с реверсом
def foldl2(f, x0, lst):
    lst.reverse()
    return foldr(lambda x,y: f(y,x), x0, lst)

In [114]:
def foldr2(f, x0, lst):
    lst.reverse()
    return foldl(lambda x,y: f(y,x), x0, lst)

In [115]:
foldl2(lambda x, y: x / y, 1, [1, 2, 3]), foldr2(lambda x, y: x / y, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

In [116]:
foldl2(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3]), foldr2(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

('123', '123')

# Задание 1.2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [99]:
# Идея алгоритма в том, что перестановка b входит в a, если подстрока a длины строки b содержит 
# все символы строки b и количество каждого символа одинаково. 
# Поэтому создается словарь, в котором подсчитывается каждый символ и сравниваются именно словари.
def check_inv(a, b):
    def make_dict(s):
        d = dict()
        for ch in s:
            if not ch in d:
                d[ch] = 1
            else: d[ch] += 1
        return d
    b_dict = make_dict(b)
    i = 0
    j = len(b) # i, j - pointers
    while j <= len(a):
        if b_dict == make_dict(a[i:j]):
            return True
        else:
            i += 1
            j += 1
    return False
# Time complexity O(m+2m(n-m)), n=len(a), b=len(b)

In [107]:
check_inv('abcrotm', 'tro'), \
check_inv('abcrtm', 'tro')

(True, False)

# Задание 1.3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [52]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        if self.left:
            yield from self.left
        if self.right:
            yield from self.right
        if not self.left and not self.right:
            yield self.value        
    
    def __str__(self):
        left = '' if not self.left else self.left.__str__()
        right = '' if not self.right else self.right.__str__()
        return f'({self.value}{left}{right})'
    
    def __repr__(self):
        return f'Tree(value={self.value}, left={self.left.__repr__()}, right={self.right.__repr__()})'
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

list(tree) == [3, 4, 2]

True

In [53]:
tree.__repr__()

'Tree(value=0, left=Tree(value=1, left=Tree(value=3, left=None, right=None), right=Tree(value=4, left=None, right=None)), right=Tree(value=2, left=None, right=None))'

In [54]:
print(tree.__str__())

(0(1(3)(4))(2))


# Задание 1.4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [16]:
def operators(stack, x): 
    op_priorities = {'+': 0, '-': 0, '*': 1, '/': 1}
    try:
        return op_priorities[x] <= op_priorities[stack[-1]]
    except KeyError:
        return False

In [17]:
import re
def calc(expr):
    expr_lst = re.findall(r'[-+/*()]|\d+(?:\d+)?', expr)
    ordered = [] 
    stack = [] 
    for x in expr_lst:
        if x.isdigit():
            ordered.append(int(x))
        elif x == '(':
            stack.append(x)
        elif x == ')':
            while stack and stack[-1] != '(':
                ordered.append(stack.pop())
            stack.pop()
        elif x in ['+', '-', '*', '/'] :
            # if x in ['+', '-', '*', '/']
            while stack and operators(stack, x):
                ordered.append(stack.pop())
            stack.append(x)
    while stack:
        ordered.append(stack.pop())
    
    # print(ordered)
    
    stack = []
    for x in ordered:
        if type(x) is int:
            stack.append(x)
        else:
            x2, x1 = stack.pop(), stack.pop()

            if x == '+':
                stack.append(x1 + x2)
            elif x == '-':
                stack.append(x1 - x2)
            elif x == '*':
                stack.append(x1 * x2)
            elif x == '/':
                stack.append(x1 // x2)
    return stack.pop()
calc('2 * (15 - 3 * 4) - 2') == 4

True

In [18]:
calc('3 * (4 - 3 * 4) - 26') == -50

True